In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import spatial
from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import dendrogram, ward

Purpose:  For the stromatolite project, identify which of the 10 flags:
    1. Is most diverse (most number of unique ID's).
    2. Is the most representitive (most similiar to others).
    3. Is the most unique (most dissimiliar to others).
    
As X inputs use and steps to clean-up data:
    1. Aggregated feature tables.
        1.1. GroupBy 20s retention time window and 20 ppm m/z.
        1.2. GroupBy flags.
        1.3. X, Y, Z = m/z, t, area
    2. GNPS ID's.
        2.1 Summarize over flags.
        2.2 Calculate pairwise similarity.
        2.3 Ming: Library search for list of id's
    3. 16S sequencing.
        3.1 Summarize over flags.
        3.2 Calculate pairwise similarity.
        
Sets = intersection, union, differences
        
Features: (Ming)
    -MZmine:
        -Aligns molecule 1 and molecule 2 (tolerance and ret time)
        -Pick average or representitive sample instead of binning
        -Assigns area and feature number
        -Rows is feature versus column
        -Could do post-clustering clean-up
        -Most number of molecules least number of smaples:
            -Set cover problem
   
   -Qiime:
       -Filter to different taxonimic levels
           X.X
           X.X.X
       -Easiest way to treat with set/cover:
           First: Pick sample with most bacteria
           Second: Pick sample with most unique bacteria
           

Part I: Genomic data

In [2]:
fn = 'GBMF classification of OTUs f_Kerry McPhail stromatolite - GBMF.unique.good.pick.good.filt.tsv'
dna16 = pd.read_csv(fn, sep="\t")

In [3]:
dna16.shape

(2375, 121)

In [4]:
dna16.head(5)

,taxlevel,rankID,taxon,daughterlevels,total,GBMF1.groups,GBMF10.groups,GBMF100.groups,GBMF101.groups,GBMF102.groups,...,GBMF90.groups,GBMF91.groups,GBMF92.groups,GBMF93.groups,GBMF94.groups,GBMF95.groups,GBMF96.groups,GBMF97.groups,GBMF98.groups,GBMF99.groups
0,0,0,Root,2,12498,1455,1576,1177,966,829,...,1163,891,1325,1196,808,1088,1314,1420,1475,793
1,1,0.1,Bacteria,50,12486,1454,1575,1177,966,829,...,1163,890,1325,1195,808,1088,1314,1420,1475,790
2,1,0.2,unknown,1,12,1,1,0,0,0,...,0,1,0,1,0,0,0,0,0,3
3,2,0.1.1,Acidobacteria,20,272,77,98,28,20,13,...,41,12,48,48,27,35,68,62,53,16
4,2,0.1.10,Calditrichaeota,1,20,0,1,0,1,1,...,0,0,0,0,0,0,0,1,0,0


In [5]:
dna16.columns

Index(['taxlevel', 'rankID', 'taxon', 'daughterlevels', 'total',
       'GBMF1.groups', 'GBMF10.groups', 'GBMF100.groups', 'GBMF101.groups',
       'GBMF102.groups',
       ...
       'GBMF90.groups', 'GBMF91.groups', 'GBMF92.groups', 'GBMF93.groups',
       'GBMF94.groups', 'GBMF95.groups', 'GBMF96.groups', 'GBMF97.groups',
       'GBMF98.groups', 'GBMF99.groups'],
      dtype='object', length=121)

In [6]:
# Correlate columne name to flag: GBMFm.groups, where m = n, nn, or nnn
# Run at different levels X.X ... X.X.X.X.X.X

In [7]:
fn = 'StromTissueOnlyGNPS042119_quant - StromTissueOnlyGNPS042119_quant.tsv'
raw_feat = pd.read_csv(fn, sep="\t")

In [8]:
raw_feat.head(5)

,row ID,row m/z,row retention time,correlation group ID,annotation network number,best ion,auto MS2 verify,identified by n=,partners,neutral M mass,...,Stromatolite_Tissue_99_pos.mzXML Peak area,Stromatolite_Tissue_91_pos.mzXML Peak area,Stromatolite_Tissue_79_pos.mzXML Peak area,Stromatolite_Tissue_95_pos.mzXML Peak area,Stromatolite_Tissue_82_pos.mzXML Peak area,Stromatolite_Tissue_80_pos.mzXML Peak area,Stromatolite_Tissue_1A_pos.mzXML Peak area,Stromatolite_Tissue_72_pos.mzXML Peak area,Stromatolite_Tissue_68_pos.mzXML Peak area,Stromatolite_Tissue_71_pos.mzXML Peak area
0,1,517.4830,5.8359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10500000.0,31700000.0,6.617186e+05,1.876589e+06,7.901106e+05,1.533882e+06,459000000.0,6.460571e+05,5.550346e+05,8.074183e+05
1,2,539.4650,5.8358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12300000.0,36200000.0,1.126247e+06,1.290824e+06,1.968257e+06,2.594918e+06,395000000.0,9.430503e+05,1.074441e+06,1.365752e+06
2,3,871.5723,13.6490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,467000000.0,41600000.0,6.360000e+07,5.840498e+04,3.170000e+08,4.170000e+07,63800000.0,1.650000e+08,9.770000e+07,4.607939e+05
3,4,537.3945,11.6977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,357000000.0,306000000.0,3.530000e+08,4.770000e+08,3.150000e+08,2.870000e+08,241000000.0,2.110000e+08,2.460000e+08,3.270000e+08
4,5,532.4393,11.6980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,255000000.0,220000000.0,2.560000e+08,3.340000e+08,2.050000e+08,1.970000e+08,173000000.0,1.630000e+08,1.660000e+08,2.450000e+08


In [9]:
raw_feat.columns[0:20]

Index(['row ID', 'row m/z', 'row retention time', 'correlation group ID',
       'annotation network number', 'best ion', 'auto MS2 verify',
       'identified by n=', 'partners', 'neutral M mass',
       'Stromatolite_Tissue_10_pos.mzXML Peak area',
       'Stromatolite_Tissue_100_pos.mzXML Peak area',
       'Stromatolite_Tissue_102_pos.mzXML Peak area',
       'Stromatolite_Tissue_103_pos.mzXML Peak area',
       'Stromatolite_Tissue_13_pos.mzXML Peak area',
       'Stromatolite_Tissue_104_pos.mzXML Peak area',
       'Stromatolite_Tissue_11_pos.mzXML Peak area',
       'Stromatolite_Tissue_101_pos.mzXML Peak area',
       'Stromatolite_Tissue_115_pos.mzXML Peak area',
       'Stromatolite_Tissue_106_pos.mzXML Peak area'],
      dtype='object')

In [10]:
# Correlate column name to flag: Stromatolite_Tissue_m_pos.mzXML, where m = n, nn, or nnn
# Bin on retention time (20s) and m/z (20 ppm), areas to float.

In [11]:
fn = 'StromatoliteTissue_UpdateReDU-MS2V3.0_July19 - ReDUMS2_sample_information_template.tsv'
meta = pd.read_csv(fn, sep="\t")

In [12]:
meta.shape

(154, 37)

In [13]:
# Unique ID's  job: RERUN Pool Region updated REDU v3 Schoenmakerskop Stromatolite Pool Samples 11/18 library search

In [14]:
# Correlate column name to flag: Stromatolite_Tissue_m_pos.mzXML, where m = n, nn, or nnn

In [15]:
fn = 'MOLECULAR-LIBRARYSEARCH-V2-402465f8-download_compound_occurrences-main.tsv'
ids = pd.read_csv(fn, sep="\t")

file_list = list(ids.columns)[2:]

for f in file_list:
    f1 = f.split('/')[-1]
    f2 = f1.split('.mzML')[0]
    ids.rename({f: f2}, axis=1, inplace=True)
    
head = list(ids.columns)[1:9]
tail = list(ids.columns)[-11:-1]
both = head + tail + ['blank_6']
ids = ids.drop(columns=both)

In [16]:
fn = 'StromatoliteTissue_UpdateReDU-MS2V3.0_July19 - ReDUMS2_sample_information_template.tsv'
meta = pd.read_csv(fn, sep="\t")
meta['filename'] = meta['filename'].str.rstrip('.mzXML')

In [17]:
ns = [1, 2, 3, 4, 5, 6, 7, 8, 10]
for n in ns:
    fs = list(meta[meta['Flag_Number'] == str(n)].filename)  
    
    # Present in redu file, but missing from Massive
    missing_files = ['Stromatolite_Tissue_16_pos', 
                    'Stromatolite_Tissue_30_pos',
                     'Stromatolite_Tissue_40_pos',
                     'Stromatolite_Tissue_46_pos',
                    'Stromatolite_Tissue_66_pos',
                    'Stromatolite_Tissue_92_pos']   
    for m in missing_files:
        if m in fs:
            fs.remove(m)
      
    label = 'site_' + str(n)
    q = ids[fs]
    ids[label] = ids[fs].sum(axis=1)
    ids.drop(columns=fs, inplace=True)

In [18]:
def f(many, unique):
    if many == 0 and unique > 0:
        return True
    else:
        return False

# Total number of ID's
number_ids = dict(zip(list(ids.columns)[1:], list(ids.sum())[1:]))

# Total number of unique ID's
ids.astype(bool).sum(axis=0)
unique_ids = dict(zip(list(ids.columns)[1:], list(ids.astype(bool).sum(axis=0))[1:]))

In [19]:
# Leave one out, calculate # unique versus other sites...
sites = ['site_1', 'site_2', 'site_3', 'site_4', 'site_5', 'site_6', 'site_7', 'site_8',
         'site_10']
sites2 = ['site_1', 'site_2', 'site_3', 'site_4', 'site_5', 'site_6', 'site_7', 'site_8',
         'site_10']
unique_vall = {}

for s in sites:
    ss = sites2
    ss.remove(s)
    ids['sums'] = ids[ss].sum(axis=1)
    ids['temp'] = ids.apply(lambda x: f(x['sums'], x[s]), axis=1)
    unique_vall[s] = ids.temp.sum()
    ss = ss.append(s)
    
ids.drop(columns=['sums', 'temp'], inplace=True)
unique_vall           

{'site_1': 19,
 'site_2': 18,
 'site_3': 63,
 'site_4': 9,
 'site_5': 15,
 'site_6': 52,
 'site_7': 24,
 'site_8': 37,
 'site_10': 36}

In [20]:
bool_ids = ids.astype(bool).astype(int).copy(deep=True).drop(columns='LibraryID')

In [21]:
# x = row, y = col [x, y]
out_arr = np.zeros((9, 9))
for x in range(0, 9):
    for y in range(0, 9):
        n = spatial.distance.cosine(bool_ids.iloc[:,x], bool_ids.iloc[:,y] )
        out_arr[x, y] = 1 - n       

cos = pd.DataFrame(out_arr).round(3)
cos.columns = ['site_1', 'site_2', 'site_3', 'site_4', 'site_5',
               'site_6', 'site_7', 'site_8', 'site_10']

cos

,site_1,site_2,site_3,site_4,site_5,site_6,site_7,site_8,site_10
0,1.000,0.479,0.448,0.521,0.438,0.464,0.513,0.464,0.442
1,0.479,1.000,0.568,0.614,0.625,0.561,0.615,0.585,0.523
2,0.448,0.568,1.000,0.553,0.545,0.643,0.610,0.640,0.640
3,0.521,0.614,0.553,1.000,0.555,0.579,0.628,0.542,0.557
4,0.438,0.625,0.545,0.555,1.000,0.532,0.603,0.605,0.551
5,0.464,0.561,0.643,0.579,0.532,1.000,0.636,0.615,0.563
6,0.513,0.615,0.610,0.628,0.603,0.636,1.000,0.653,0.603
7,0.464,0.585,0.640,0.542,0.605,0.615,0.653,1.000,0.630
8,0.442,0.523,0.640,0.557,0.551,0.563,0.603,0.630,1.000


In [22]:
cos_sum = dict(cos.sum())
cos_mean = dict(cos.mean())

out_df = pd.DataFrame()
out_df['sites'] = number_ids.keys()
out_df['number_ids'] = number_ids.values()
out_df['unique_ids'] = unique_ids.values()
out_df['unique_vall'] = unique_vall.values()
out_df['cos_sum'] = cos_sum.values()
out_df['cos_mean'] = cos_mean.values()
out_df.set_index('sites')
out_df.round(2)

,sites,number_ids,unique_ids,unique_vall,cos_sum,cos_mean
0,site_1,460,150,19,4.77,0.53
1,site_2,621,241,18,5.57,0.62
2,site_3,1632,417,63,5.65,0.63
3,site_4,524,186,9,5.55,0.62
4,site_5,725,239,15,5.45,0.61
5,site_6,934,342,52,5.59,0.62
6,site_7,936,295,24,5.86,0.65
7,site_8,1334,354,37,5.73,0.64
8,site_10,949,302,36,5.51,0.61


In [23]:
# To do:  Analysis by features, and analysis by genomic stuff!

In [24]:
raw_feat.shape

(11384, 140)

In [25]:
x = list(raw_feat.columns)
y = x[0:3]
z = x[-130:-1]
u = y + z

In [26]:
proc_feat = raw_feat[u].copy(deep=True)

In [27]:
names = list(proc_feat.columns)[3:]

In [28]:
replace_dict = {}
for name in names:
    n = name.split('.')[0]
    replace_dict[name] = n  
    
proc_feat = proc_feat.rename(columns=replace_dict).copy(deep=True)

In [29]:
present_files = list(proc_feat.columns)
final_feat = proc_feat.iloc[:,0:3].copy(deep=True)

In [30]:
flags = ['1', '2', '3', '4', '5', '6', '7', '8', '10']
for f in flags:
    col_to_pick = list(meta[meta.Flag_Number == f].filename)
    
    # Present in redu file, but missing from Massive
    missing_files = ['Stromatolite_Tissue_16_pos', 
                    'Stromatolite_Tissue_30_pos',
                     'Stromatolite_Tissue_40_pos',
                     'Stromatolite_Tissue_46_pos',
                    'Stromatolite_Tissue_66_pos',
                    'Stromatolite_Tissue_92_pos']   
    for m in missing_files:
        if m in col_to_pick:
            col_to_pick.remove(m)
            
    checked_col_to_pick = []
    for c in col_to_pick:
        if c in present_files:
            checked_col_to_pick.append(c)
    
    out_title = 'sum_' + f

    final_feat[out_title] = proc_feat[checked_col_to_pick].sum(axis=1, 
                                                               skipna=True,
                                                              numeric_only=True)

In [138]:
final_feat

,row ID,row m/z,row retention time,sum_1,sum_2,sum_3,sum_4,sum_5,sum_6,sum_7,sum_8,sum_10
0,1,517.4830,5.8359,7.015500e+09,1.541656e+07,6.136707e+09,1.551042e+07,4.658652e+06,1.949041e+07,8.886632e+06,2.126596e+09,1.229202e+09
1,2,539.4650,5.8358,5.185800e+09,1.762364e+07,4.486369e+09,1.974122e+07,4.931948e+06,2.186030e+07,1.345746e+07,2.109378e+09,1.347400e+09
2,3,871.5723,13.6490,3.371560e+09,2.580225e+09,7.907935e+09,2.596622e+09,3.512200e+09,2.079400e+09,3.767300e+09,3.364949e+09,4.852518e+09
3,4,537.3945,11.6977,3.978000e+09,3.715000e+09,8.377000e+09,3.310000e+09,4.006000e+09,2.980000e+09,3.852000e+09,5.886000e+09,4.576000e+09
4,5,532.4393,11.6980,2.696000e+09,2.556000e+09,5.742000e+09,2.333000e+09,2.802000e+09,2.054000e+09,2.714000e+09,4.192000e+09,3.200000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
11379,38523,311.2220,4.5883,1.085796e+06,7.079029e+05,1.211532e+07,1.039420e+06,3.795534e+06,4.747803e+06,1.115058e+07,5.278053e+06,6.156807e+06
11380,38534,344.2801,3.9611,1.710442e+03,4.220484e+04,5.449430e+06,1.577832e+05,3.654567e+05,1.984119e+06,1.254403e+06,1.655166e+06,2.008885e+06
11381,38539,235.2058,4.5840,3.092209e+06,6.696389e+05,1.597207e+07,2.948499e+06,6.505277e+06,8.677586e+06,7.134641e+06,8.167086e+06,1.735757e+07
11382,38752,886.5476,5.9459,0.000000e+00,5.095750e+05,1.679539e+06,4.493016e+05,1.222971e+06,1.675469e+06,3.982267e+05,1.594338e+06,1.196960e+06


In [141]:
final_feat.astype('bool').sum(axis=0)

row ID                11384
row m/z               11384
row retention time    11384
sum_1                 10417
sum_2                 10763
sum_3                 11247
sum_4                 10793
sum_5                 10861
sum_6                 10939
sum_7                 10944
sum_8                 11095
sum_10                10995
dtype: int64

In [32]:
calc_feat = final_feat.iloc[:,3:]

In [33]:
# To calculate:
# Number of ID's
# Unique ID's versus all
# Sum intensity
# Similarity matrix versus other samples, avg and median.

In [34]:
num_ids = dict(calc_feat.astype(bool).sum(axis=0))

In [35]:
area_sum = dict(calc_feat.sum())

In [36]:
cols = list(calc_feat.columns)

In [37]:
print(cols)

['sum_1', 'sum_2', 'sum_3', 'sum_4', 'sum_5', 'sum_6', 'sum_7', 'sum_8', 'sum_10']


In [38]:
unique_dict = {}
for c in cols:
    # Select non-zero, feature present, in specified column.
    working = calc_feat[calc_feat[c] > 0].copy(deep=True)
    
    # Calculate the sum of area in all other columns.
    working['sum_others'] = working[cols].sum(axis=1) - working[c]
    
    not_unique = working['sum_others'].astype(bool).sum(axis=0)
    unique = list(working.shape)[0] - not_unique
    unique_dict[c] = unique
    

In [39]:
unique_dict

{'sum_1': 1,
 'sum_2': 1,
 'sum_3': 5,
 'sum_4': 0,
 'sum_5': 1,
 'sum_6': 3,
 'sum_7': 1,
 'sum_8': 4,
 'sum_10': 2}

In [40]:
# Similarity matrix, copy code from above?

In [44]:
final_feat

,row ID,row m/z,row retention time,sum_1,sum_2,sum_3,sum_4,sum_5,sum_6,sum_7,sum_8,sum_10
0,1,517.4830,5.8359,7.015500e+09,1.541656e+07,6.136707e+09,1.551042e+07,4.658652e+06,1.949041e+07,8.886632e+06,2.126596e+09,1.229202e+09
1,2,539.4650,5.8358,5.185800e+09,1.762364e+07,4.486369e+09,1.974122e+07,4.931948e+06,2.186030e+07,1.345746e+07,2.109378e+09,1.347400e+09
2,3,871.5723,13.6490,3.371560e+09,2.580225e+09,7.907935e+09,2.596622e+09,3.512200e+09,2.079400e+09,3.767300e+09,3.364949e+09,4.852518e+09
3,4,537.3945,11.6977,3.978000e+09,3.715000e+09,8.377000e+09,3.310000e+09,4.006000e+09,2.980000e+09,3.852000e+09,5.886000e+09,4.576000e+09
4,5,532.4393,11.6980,2.696000e+09,2.556000e+09,5.742000e+09,2.333000e+09,2.802000e+09,2.054000e+09,2.714000e+09,4.192000e+09,3.200000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
11379,38523,311.2220,4.5883,1.085796e+06,7.079029e+05,1.211532e+07,1.039420e+06,3.795534e+06,4.747803e+06,1.115058e+07,5.278053e+06,6.156807e+06
11380,38534,344.2801,3.9611,1.710442e+03,4.220484e+04,5.449430e+06,1.577832e+05,3.654567e+05,1.984119e+06,1.254403e+06,1.655166e+06,2.008885e+06
11381,38539,235.2058,4.5840,3.092209e+06,6.696389e+05,1.597207e+07,2.948499e+06,6.505277e+06,8.677586e+06,7.134641e+06,8.167086e+06,1.735757e+07
11382,38752,886.5476,5.9459,0.000000e+00,5.095750e+05,1.679539e+06,4.493016e+05,1.222971e+06,1.675469e+06,3.982267e+05,1.594338e+06,1.196960e+06


In [49]:
proc_feat = final_feat.copy(deep=True)

In [51]:
proc_feat.drop(columns=['row ID', 'row m/z', 'row retention time']).astype(int)

,sum_1,sum_2,sum_3,sum_4,sum_5,sum_6,sum_7,sum_8,sum_10
0,7015500000,15416562,6136707321,15510417,4658651,19490412,8886631,2126595679,1229201542
1,5185800000,17623637,4486369268,19741217,4931948,21860302,13457457,2109378309,1347400000
2,3371559652,2580224794,7907934605,2596622085,3512199959,2079400000,3767300000,3364949401,4852518292
3,3978000000,3715000000,8377000000,3310000000,4006000000,2980000000,3852000000,5886000000,4576000000
4,2696000000,2556000000,5742000000,2333000000,2802000000,2054000000,2714000000,4192000000,3200000000
...,...,...,...,...,...,...,...,...,...
11379,1085795,707902,12115316,1039420,3795534,4747802,11150577,5278052,6156806
11380,1710,42204,5449430,157783,365456,1984118,1254402,1655166,2008884
11381,3092208,669638,15972070,2948498,6505276,8677586,7134640,8167086,17357571
11382,0,509575,1679538,449301,1222971,1675469,398226,1594337,1196959


In [53]:
# x = row, y = col [x, y]
out_arr = np.zeros((9, 9))
for x in range(0, 9):
    for y in range(0, 9):
        n = spatial.distance.cosine(proc_feat.iloc[:,x], proc_feat.iloc[:,y] )
        out_arr[x, y] = 1 - n       

cos_feat = pd.DataFrame(out_arr).round(3)
cos_feat.columns = ['site_1', 'site_2', 'site_3', 'site_4', 'site_5',
               'site_6', 'site_7', 'site_8', 'site_10']

cos_feat

,site_1,site_2,site_3,site_4,site_5,site_6,site_7,site_8,site_10
0,1.000,0.739,0.722,0.031,0.051,0.071,0.046,0.059,0.093
1,0.739,1.000,0.951,0.121,0.174,0.190,0.138,0.159,0.170
2,0.722,0.951,1.000,0.131,0.179,0.203,0.138,0.165,0.172
3,0.031,0.121,0.131,1.000,0.481,0.724,0.339,0.428,0.323
4,0.051,0.174,0.179,0.481,1.000,0.735,0.802,0.882,0.452
5,0.071,0.190,0.203,0.724,0.735,1.000,0.558,0.691,0.593
6,0.046,0.138,0.138,0.339,0.802,0.558,1.000,0.953,0.432
7,0.059,0.159,0.165,0.428,0.882,0.691,0.953,1.000,0.511
8,0.093,0.170,0.172,0.323,0.452,0.593,0.432,0.511,1.000


In [55]:
cos_feat_sum = dict(cos_feat.sum())
cos_feat_mean = dict(cos_feat.mean())

out_feat_df = pd.DataFrame()
out_feat_df['sites'] = number_ids.keys()
out_feat_df['number_ids'] = number_ids.values()
out_feat_df['area_sum'] = area_sum.values()
out_feat_df['unique_dict'] = unique_vall.values()
out_feat_df['cos_sum'] = cos_feat_sum.values()
out_feat_df['cos_mean'] = cos_feat_mean.values()
out_feat_df.set_index('sites')
out_feat_df.round(2)

,sites,number_ids,area_sum,unique_dict,cos_sum,cos_mean
0,site_1,460,1.630005e+11,19,2.81,0.31
1,site_2,621,2.288671e+11,18,3.64,0.40
2,site_3,1632,5.956921e+11,63,3.66,0.41
3,site_4,524,2.255433e+11,9,3.58,0.40
4,site_5,725,2.678990e+11,15,4.76,0.53
5,site_6,934,3.846793e+11,52,4.76,0.53
6,site_7,936,4.560875e+11,24,4.41,0.49
7,site_8,1334,5.713464e+11,37,4.85,0.54
8,site_10,949,3.936328e+11,36,3.75,0.42


In [41]:
# Genetic differences...

In [56]:
dna16.head(5)

,taxlevel,rankID,taxon,daughterlevels,total,GBMF1.groups,GBMF10.groups,GBMF100.groups,GBMF101.groups,GBMF102.groups,...,GBMF90.groups,GBMF91.groups,GBMF92.groups,GBMF93.groups,GBMF94.groups,GBMF95.groups,GBMF96.groups,GBMF97.groups,GBMF98.groups,GBMF99.groups
0,0,0,Root,2,12498,1455,1576,1177,966,829,...,1163,891,1325,1196,808,1088,1314,1420,1475,793
1,1,0.1,Bacteria,50,12486,1454,1575,1177,966,829,...,1163,890,1325,1195,808,1088,1314,1420,1475,790
2,1,0.2,unknown,1,12,1,1,0,0,0,...,0,1,0,1,0,0,0,0,0,3
3,2,0.1.1,Acidobacteria,20,272,77,98,28,20,13,...,41,12,48,48,27,35,68,62,53,16
4,2,0.1.10,Calditrichaeota,1,20,0,1,0,1,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Show overall table at 1 and 2 (cyanos)
# Cos similarity 
# Sum at taxon level 0-n

In [92]:
names = list(dna16.columns)[5:]

replace_dict = {}
for name in names:
    n = name.split('GBMF')[1].split('.')[0]
    n = 'Stromatolite_Tissue_' + n + '_pos'
    replace_dict[name] = n  
    
proc_dna16 = dna16.rename(columns=replace_dict).copy(deep=True)

ns = [1, 2, 3, 4, 5, 6, 7, 8, 10]
for n in ns:
    fs = list(meta[meta['Flag_Number'] == str(n)].filename)  

    # Present in redu file, but missing from Massive

    missing_files = ['Stromatolite_Tissue_1D_pos', 'Stromatolite_Tissue_1C_pos', 
    'Stromatolite_Tissue_1A_pos', 'Stromatolite_Tissue_2B-R_pos', 
    'Stromatolite_Tissue_2B-L_pos', 'Stromatolite_Tissue_2C-L_pos', 
    'Stromatolite_Tissue_2E-R_pos', 'Stromatolite_Tissue_2D-R_pos', 
    'Stromatolite_Tissue_2E-L_pos', 'Stromatolite_Tissue_2D-L_pos', 
    'Stromatolite_Tissue_2A-R_pos', 'Stromatolite_Tissue_1B_pos', 
    'Stromatolite_Tissue_2C-R_pos', 'Stromatolite_Tissue_1E_pos', 
    'Stromatolite_Tissue_2A-L_pos']   
    
    for m in missing_files:
        if m in fs:
            fs.remove(m)

    label = 'site_' + str(n)

    q = proc_dna16[fs]
    
    proc_dna16[label] = q.sum(axis=1)
    proc_dna16.drop(columns=fs, inplace=True)


In [91]:
proc_dna16

,taxlevel,rankID,taxon,daughterlevels,total,site_1,site_2,site_3,site_4,site_5,site_6,site_7,site_8,site_10
0,0,0,Root,2,12498,17792,20516,16788,19263,18830,19643,18451,19883,14539
1,1,0.1,Bacteria,50,12486,17788,20502,16780,19252,18816,19640,18448,19880,14535
2,1,0.2,unknown,1,12,4,14,8,11,14,3,3,3,4
3,2,0.1.1,Acidobacteria,20,272,1057,738,538,706,640,715,746,762,433
4,2,0.1.10,Calditrichaeota,1,20,11,27,6,28,26,9,31,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,6,0.1.7.6.2.1.4,uncultured,0,21,19,6,22,24,26,59,41,16,7
2371,6,0.1.8.1.1.1.1,CK-2C2-2_ge,0,2,0,4,3,2,3,1,0,2,5
2372,6,0.1.9.1.1.1.1,WCHB1-02_ge,0,2,2,0,1,0,0,0,0,4,5
2373,6,0.2.1.1.1.1.1,unclassified,0,1,0,0,0,0,1,0,0,0,1


In [110]:
proc_dna16[proc_dna16.taxlevel==2]

,taxlevel,rankID,taxon,daughterlevels,total,site_1,site_2,site_3,site_4,site_5,site_6,site_7,site_8,site_10
3,2,0.1.1,Acidobacteria,20,272,1057,738,538,706,640,715,746,762,433
4,2,0.1.10,Calditrichaeota,1,20,11,27,6,28,26,9,31,1,6
5,2,0.1.11,Chlamydiae,2,197,125,248,135,220,129,125,100,186,162
6,2,0.1.12,Chloroflexi,13,795,1339,1570,925,1415,1282,1155,1316,1099,906
7,2,0.1.13,Cyanobacteria,4,462,671,1184,1163,1029,1188,1055,1084,1263,933
8,2,0.1.14,Dadabacteria,1,6,17,12,8,12,10,21,24,18,5
9,2,0.1.15,Deinococcus-Thermus,1,10,31,2,13,5,11,25,11,36,9
10,2,0.1.16,Dependentiae,1,229,138,244,169,196,199,115,170,145,163
11,2,0.1.17,Elusimicrobia,6,83,184,146,71,82,93,90,78,97,69
12,2,0.1.18,Entotheonellaeota,1,5,31,2,10,8,10,17,16,8,1


In [121]:
cos_dna = proc_dna16.copy(deep=True)

In [124]:
cos_dna = cos_dna.drop(columns=['taxlevel', 'rankID', 'taxon', 'daughterlevels', 'total']).astype(int).copy(deep=True)

In [126]:
# x = row, y = col [x, y]
out_arr = np.zeros((9, 9))
for x in range(0, 9):
    for y in range(0, 9):
        n = spatial.distance.cosine(cos_dna.iloc[:,x], cos_dna.iloc[:,y] )
        out_arr[x, y] = 1 - n       

cos_dna = pd.DataFrame(out_arr).round(3)
cos_dna.columns = ['site_1', 'site_2', 'site_3', 'site_4', 'site_5',
               'site_6', 'site_7', 'site_8', 'site_10']

cos_dna

,site_1,site_2,site_3,site_4,site_5,site_6,site_7,site_8,site_10
0,1.000,0.998,0.997,0.997,0.997,0.997,0.997,0.997,0.996
1,0.998,1.000,0.998,1.000,0.999,0.998,0.999,0.998,0.999
2,0.997,0.998,1.000,0.998,0.999,0.999,0.999,0.999,0.999
3,0.997,1.000,0.998,1.000,1.000,0.999,0.999,0.998,0.998
4,0.997,0.999,0.999,1.000,1.000,0.999,1.000,0.998,0.998
5,0.997,0.998,0.999,0.999,0.999,1.000,0.999,0.999,0.997
6,0.997,0.999,0.999,0.999,1.000,0.999,1.000,0.999,0.998
7,0.997,0.998,0.999,0.998,0.998,0.999,0.999,1.000,0.998
8,0.996,0.999,0.999,0.998,0.998,0.997,0.998,0.998,1.000


In [134]:
un_dna = proc_dna16.copy(deep=True)
un_dna = un_dna.drop(columns=['taxlevel', 'rankID', 'taxon', 'daughterlevels', 'total']).astype(int).copy(deep=True)

In [135]:
un_dna

,site_1,site_2,site_3,site_4,site_5,site_6,site_7,site_8,site_10
0,17792,20516,16788,19263,18830,19643,18451,19883,14539
1,17788,20502,16780,19252,18816,19640,18448,19880,14535
2,4,14,8,11,14,3,3,3,4
3,1057,738,538,706,640,715,746,762,433
4,11,27,6,28,26,9,31,1,6
...,...,...,...,...,...,...,...,...,...
2370,19,6,22,24,26,59,41,16,7
2371,0,4,3,2,3,1,0,2,5
2372,2,0,1,0,0,0,0,4,5
2373,0,0,0,0,1,0,0,0,1


In [136]:
unique_dict = {}
cols = un_dna.columns
for c in cols:
    # Select non-zero, feature present, in specified column.
    working = un_dna[un_dna[c] > 0].copy(deep=True)
    
    # Calculate the sum of area in all other columns.
    working['sum_others'] = working[cols].sum(axis=1) - working[c]
    
    not_unique = working['sum_others'].astype(bool).sum(axis=0)
    unique = list(working.shape)[0] - not_unique
    unique_dict[c] = unique

In [137]:
unique_dict

{'site_1': 48,
 'site_2': 29,
 'site_3': 17,
 'site_4': 38,
 'site_5': 9,
 'site_6': 40,
 'site_7': 10,
 'site_8': 3,
 'site_10': 14}